In [ ]:
#ROUNDABOUT SIMULATION

#GLOBAL VARIABLES

N = "North"
S = "South"
E = "East"
W = "West"
MEAN_ARRIVAL_TIME = 10
PRINT_EVENTS = True

class Node:
    def __init__(self, value):
        self.value = None
        self.next = None
        self.prev = None

class CDLL:
    def __init__(self):
        self.head = None
        self.tail = None
    def makeCDLL(self):
        new_node = Node(0)
        new_node.next = None
        new_node.prev = None
        self.head = new_node
        self.tail = new_node
    def insert(self, new_node):
        temp = self.tail
        self.tail.next = new_node
        self.tail = new_node
        new_node.next = self.head
        new_node.prev = temp
        self.head.prev = new_node
        
class RoundaboutMkr:
    def __init__(self, num_of_cars):
        self.num_of_cars = num_of_cars
        Roundabout = CDLL()
        Roundabout.makeCDLL()
        car_index = 1
        while(car_index <= num_of_cars):
            new_node = Node(car_index)
            Roundabout.insert(new_node)
            car_index += 1

    def PrintRoundabout(self):
        current = Roundabout.head
        while(current.value != None):
            if(current == Roundabout.tail):
                break
            print("car value" + current.value + "\n")
            current = current.next

class Car:
    
    stop_time = 5
    roundabout_time = 5

    #each car has its own index, arrival time, destination
    def __init__(self, index, arrival_time, dest):
        self.index = index
        self.arrival_time = arrival_time
        self.dest = dest
        
    #Returns driver instance stop time
    def get_stop_time(self):
        return self.stop_time

    #Returns driver instance time to traverse to next spot in roundabout
    def get_round_time(self):
        return self.roundabout_time 
    

class Pedestrian:

    #each pedestrian has its own arrival time, destination, and time to cross street
    def __init__(self, arrival_time, dest, cross_time):
        self.arrival_time = arrival_time
        self.dest = dest
        self.cross_time = cross_time

class Event:

    def __init__(self, event_type, time, dest):
        self.type = event_type
        self.time = time
        self.dest = dest

class EventQueue:

    def __init__(self):
        self.events = []

    #Add event (will get sent to the back of the queue)
    def add_event(self, event):
        #print("Adding event: " + event.type + ", clock: " + str(event.time))
        self.events.append(event)

    #Get the next event in the queue and pop it (remove it)
    #Returns removed next event
    def get_next_event(self):
        min_time = 9999999999999
        min_index = 0
        for i in range(len(self.events)):
            if self.events[i].time < min_time:
                min_time = self.events[i].time
                min_index = i
        event = self.events.pop(min_index)
        #print("Removing event: " + event.type + ", clock: " + str(event.time))
        return event

class Simulation:

    upper_arrival_time = 2 * MEAN_ARRIVAL_TIME

    def __init__(self, total_arrivals):
        self.arrivals = 0
        self.total_arrivals = total_arrivals
        self.clock = 0

        #queues waiting to enter roundabout
        self.north, self.east, self.south, self.west = [], [], [], []
        self.north_ready = False
        self.east_ready = False
        self.south_ready = False
        self.west_ready = False
        self.intersection_free = True
        self.events = EventQueue()
        self.generate_arrival()

        #queues in roundabout: modelled with circle linked lists
        self.left_straight = RoundaboutMkr(12)      #make inner roundabout with 12 possible cars
        self.right = RoundaboutMkr(16)              #make outer roundabout with 16 possible cars

        #enables us to see what is going on
        self.print_events = PRINT_EVENTS
        #saves our data for arrival time of 4*3*2 + 4 = 28 cases
        self.self_driv = [[0]*3]*4      #self driving data
        self.humn_driv = [[0]*3]*4      #human driver data
        self.ped = [0]*4                #pedestrian data

    def run(self):
        while self.arrivals <= self.total_arrivals:
            if self.print_events:
                self.print_state()
            self.execute_next_event()

    '''
    EVENT TYPES:
    Arrive at intersection - ARRIVAL
    Stop at intersection - STOP_INT
    Enter the roundabout - ENTER
    Traverse to next spot in roundabout - GO_ROUND
    Stop in roundabout - STOP_ROUND; though we don't need this if we assume all cars entering roundabout will yield until there is an empty spot??
        - ASSUME THE ABOVE AT FIRST!!
    Depart from simulation - DEPARTURE
    Collision - COLLISION
    '''

    #Execute the next event in the queue
    #(Get next event, and execute appropriate method depending on event type)
    def execute_next_event(self):
        event = self.events.get_next_event()
        self.clock = event.time
        if event.type == ARRIVAL:
            self.execute_arrival(event)
        if event.type == DEPARTURE:
            self.execute_departure(event)
        if event.type == STOP_INT:
            self.execute_stop_at_intersection(event)
        if event.type == ENTER:
            self.execute_enter(event)
        if event.type == GO_ROUND:
            self.execute_traverse_roundabout(event)
        if event.type == COLLISION:
            self.execute_collision(event)

    def execute_departure(event):
        #wherever the node is in the list, make the element 0

    def execute_stop_at_intersection(event):

    def execute_enter(event):

    def execute_traverse_roundabout(event):
    
    def execute_collision(event):


    #Start arrival event 
    def execute_arrival(self, event):
        r = random.random()
        #cases where destination is N: r < 0.3 && dir = W, r < 0.6 && dir = E, or dir = N
        #cases where destination is S: r < 0.3 && dir = E, r < 0.6 && dir = W, or dir = S
        #cases where destination is E: r < 0.3 && dir = N, r < 0.6 && dir = S, or dir = E
        #cases where destination is W: r < 0.3 && dir = S, r < 0.6 && dir = N, or dir = E
        if (r < 0.3 and event.direction == W) or (r >= 0.3 and r < 0.6 and event.direction == E) or (r >= 0.6 and event.direction == N):
            driver = Driver(self.num_of_arrivals, self.clock, N)
        elif (r < 0.3 and event.direction == E) or (r >= 0.3 and r < 0.6 and event.direction == W) or (r >= 0.6 and event.direction == S):
            driver = Driver(self.num_of_arrivals, self.clock, S)
        elif (r < 0.3 and event.direction == N) or (r >= 0.3 and r < 0.6 and event.direction == S) or (r >= 0.6 and event.direction == E):
            driver = Driver(self.num_of_arrivals, self.clock, E)
        else:
            driver = Driver(self.num_of_arrivals, self.clock, W)
        if self.print_events:
            print(str(self.clock)+ ": A driver arrives from the " + event.direction + ".")

        if event.direction == N:
            if self.north == []: #Car needs to stop before clearing
                self.north_ready = False
            self.north.append(driver)
            new_event = Event(STOP, self.clock + driver.get_stop_time(), N)
            self.events.add_event(new_event)
            
        elif event.direction == E:
            if self.east == []: #Car needs to stop before clearing
                self.east_ready = False
            self.east.append(driver)
            new_event = Event(STOP, self.clock + driver.get_stop_time(), E)
            self.events.add_event(new_event)
            
        elif event.direction == S:
            if self.south == []: #Car needs to stop before clearing
                self.south_ready = False
            self.south.append(driver)
            new_event = Event(STOP, self.clock + driver.get_stop_time(), S)
            self.events.add_event(new_event)
            
        else:
            if self.west == []: #Car needs to stop before clearing
                self.west_ready = False
            self.west.append(driver)
            new_event = Event(STOP, self.clock + driver.get_stop_time(), W)
            self.events.add_event(new_event)
            
        self.generate_arrival() #Generate the next arrival

    #Generate a car arriving at the intersection
    def generate_arrival(self):
        #Generates a random number uniformily between 0 and upper_arrival_time
        inter_arrival_time = random.random() * self.upper_arrival_time
        time = self.clock + inter_arrival_time
        
        r = random.random()
        #Equally likely to arrive from each direction
        if r < 0.25:
            self.events.add_event(Event(ARRIVAL, time, N))
        elif r < 0.5:
            self.events.add_event(Event(ARRIVAL, time, E))
        elif r < 0.75:
            self.events.add_event(Event(ARRIVAL, time, S))
        else:
            self.events.add_event(Event(ARRIVAL, time, W))
        self.num_of_arrivals += 1 #Needed for the simulation to terminate

    def print_state(self):
        print("[N,E,S,W] = ["+ str(len(self.north)) + ","+ str(len(self.east)) +","+ str(len(self.south)) +","+ str(len(self.west)) +"]")

        
